In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
import keras
import pickle

Using TensorFlow backend.
/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/xuny

In [2]:
import os

data_dir="./data/"
ticker="TSLA"

date_pool=pd.date_range("1/1/2019","1/31/2019",freq="B").strftime("%Y%m%d")
date_pool=[d for d in date_pool if os.path.exists(data_dir+"trades_{}_{}.csv".format(d,ticker))]

train_days=10
train_date_list=date_pool[:train_days]
test_date_list=date_pool[train_days+1:]
time_steps = 50

nforward=10

In [135]:
class Pipeline():
    def __init__(self):
        self.x = None
        self.y = None
        self.scaler = None
    def load_data(self, ticker, date):
        df = pd.read_csv(data_dir+'trades_{}_{}.csv'.format(date, ticker),index_col=[0],parse_dates=[0])

        # Feature Engineering
        df["direction"]=(df["trade_px"]-df["trade_px"].shift(1)).apply(np.sign)
        df["pct_change"]=df["trade_px"].pct_change()

        mysign=lambda x: 0 if abs(x)<1e-5 else (1 if x>0 else -1)
        df["label"]=(df["trade_px"].rolling(nforward).mean().shift(-nforward)-df["trade_px"]).apply(mysign)
        # df["label"]=(df["trade_px"].shift(-1)-df["trade_px"]).apply(np.sign) # last version

        df.fillna(method="ffill",inplace=True)
        df.dropna(axis=0,inplace=True)
        # print(df.head(10),df.shape)
        # print("NaN number: ",df.isna().sum().sum())

        return df[["trade_px","trade_size","pct_change","direction","label"]].values

    def create_dataset(self, ticker=ticker, dates=train_date_list, time_steps = time_steps, input_scaler=None):  
        for i,d in enumerate(dates):
            datanew = self.load_data(ticker,d)
            if i==0:
                data=datanew
            else:
                data=np.vstack((data, datanew))

        label=data[:,-1]
        data=data[:,:-1]

        if input_scaler is None:
            scaler=StandardScaler()
            data=scaler.fit_transform(data)
        else:
            data=input_scaler.transform(data)
            scaler=input_scaler

        x = [data[0 : time_steps]]
        y = [label[time_steps-1]]
        N=len(data)//time_steps

        print(N)
        for i in range(1, N):
            t = data[i*time_steps: (i + 1)*time_steps]
            x = np.vstack((x, [t]))
            y.append(label[(i + 1)*time_steps-1])

        y=pd.get_dummies(y)
        #print(y)

        return x,y.values,scaler

    def loss_plot(self, history, plot_name = 'Loss'): # type(history) is dict
        loss = np.asarray(history['loss'])
        val_loss = np.asarray(history['val_loss'])

        plt.style.use('seaborn')
        plt.figure(figsize = (20,6), dpi=dpi)
        plt.grid(True)
        plt.plot(loss, color = 'darkgrey')
        plt.plot(val_loss, color = 'tan')
        plt.legend(['loss', 'val_loss'])
        # plt.savefig('{}_{}_{}_{}_{}.png'.format(ticker, plot_name, str(n_epochs), str(time_steps), str(batch_size)))
    
    
    def training_data_transform(self, ticker):
        # Load train data
        x, y, scaler = self.create_dataset(ticker)
        self.x, self.y, self.scaler = x, y, scaler
        print("Finished loading data.")

        with open("model/LSTMv2_scaler_{}_{}.p".format(train_date_list[0],train_date_list[-1]),"wb") as f:
            pickle.dump(scaler,f)

    def model_training_testing(self, ticker, model, plot = False):
        # Model Training pipeline
        model_functionalities = Model_Functionalities(model)
        
        x, y, scaler = self.x, self.y, self.scaler
        
        if x is None:
            print("None Training data processed")
            return
        
        # Build model, in-sample train test
        train_history = model_functionalities.train_test(x, y, plot)  
        if model_functionalities.model.model_name == "LSTM":
            if plot == True:
                self.loss_plot(train_history.history)

        with open("model/LSTMv2_{}_{}.p".format(train_date_list[0],train_date_list[-1]),"wb") as f:
            pickle.dump(model,f)

        # Out-of-sample test
        for test_date in test_date_list:
            # create test dateset
            x_test, y_test, _ = self.create_dataset(ticker=ticker, dates=[test_date], time_steps = time_steps, input_scaler=scaler)
            x_test, y_test = model_functionalities.model.reshape_dataset(x_test, y_test)

            # use precious trained model to test
            y_test_pred = model_functionalities.predict(x_test)
            if model_functionalities.model.model_name == "LSTM":
                if plot == True:
                    model_functionalities.view_accuracy(y_test_pred.argmax(axis=1), y_test.argmax(axis=1))
            if y_test.shape[1] != 1:
                accuracy = np.mean(y_test_pred.argmax(axis=1)==y_test.argmax(axis=1))
            else:
                accuracy = np.mean(y_test_pred==y_test)
            print(test_date+" accuracy: ", accuracy)
        return model_functionalities.model

In [136]:
pipeline = Pipeline()
pipeline.training_data_transform(ticker)

15154
Finished loading data.


# Using LSTM to predict next up and down
## v2: Predict up and downs of the average of $\mathbf{nforward}=10$ following prices

1. Too slow to predict (next trade may happen in millisecond)
2. Only classification of up, down and same. No quantitative prediction (can be improved to predict quantity of price movement)

In [167]:
hidden_dim = 30
n_epochs = 100
batch_size = 128
activation = "tanh"
loss = 'categorical_crossentropy'
stop_patience=20

dpi=200


class LSTM_Model():
    def __init__(self):
        self.model = Sequential()
        self.model_name = "LSTM"
        return 
    
    def build(self,  time_steps = time_steps, data_dim = 1, output_dim = 3):
        # expected input batch shape: (batch_size, timesteps, data_dim)
        # the sample of index i in batch k is the follow-up for the sample i in batch k-1.
        self.model.add(LSTM(hidden_dim, activation=activation, return_sequences=True, input_shape=(time_steps, data_dim)))
        self.model.add(LSTM(hidden_dim, activation=activation, return_sequences=True))
        self.model.add(LSTM(hidden_dim, activation=activation))
        self.model.add(Dense(output_dim, activation = 'softmax'))
        
        opt=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        self.model.compile(loss = loss, optimizer=opt, metrics=['accuracy']) 
        return self.model
    
    def fit(self, x_train, y_train, batch_size, epochs, validation_data, callbacks):
        return self.model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = validation_data, callbacks = callbacks)
    
    def predict(self, x):
        return self.model.predict(x)
    
    def reshape_dataset(self, x, y):
        if x is not None:
            if len(x.shape) == 2:
                x = x.reshape(x.shape[0], x.shape[1], 1)
        if len(y.shape) == 1:
            y = y.reshape(y.shape[0],1)
        return x, y 


class Model_Functionalities():
    def __init__(self, model):
        self.model = model
    def train_test(self, x, y, plot = False):
        
        size = len(x)
        if size!=len(y):
            return None
        x = x[: batch_size * (size // batch_size)]
        y = y[: batch_size * (size // batch_size)]
        
        x, y = self.model.reshape_dataset(x, y)

        x_train, x_validation, y_train, y_validation= train_test_split(x, y, test_size = 0.1, shuffle = False)
        print('train', x_train.shape, y_train.shape)
        print('validation', x_validation.shape, y_validation.shape)
        
        
        if self.model.model_name == "LSTM":
            early_stopping = EarlyStopping(monitor='val_loss', patience=stop_patience, mode="min", verbose=2, restore_best_weights=True)
            history = self.model.fit(x_train, y_train, batch_size = batch_size, epochs = n_epochs,
                                     validation_data=(x_validation, y_validation),callbacks=[early_stopping])
        else:
            self.model.fit(x_train, y_train)
        
        self.y_pred = self.model.predict(x_validation)
        self.y_validation_true = y_validation
        
        if plot == True:
            if y.shape[1] != 1:
                self.train_plot = self.view_accuracy(self.predict(x_train).argmax(axis=1), y_train.argmax(axis=1), 'Train')
                self.validation_plot = self.view_accuracy(self.predict(x_validation).argmax(axis=1), y_validation.argmax(axis=1), 'Validation')
            else:
                self.train_plot = self.view_accuracy(self.predict(x_train), y_train, 'Train')
                self.validation_plot = self.view_accuracy(self.predict(x_validation), y_validation, 'Validation')
        if self.model.model_name == "LSTM":
            return history

    def predict(self, x_validation):
        pred = self.model.predict(x_validation)
        return pred
    
    def view_accuracy(self, y_pred = None, y_true = None, plot_name = 'Test', num=100):
        if y_pred is None:
            y_pred = self.y_pred.argmax(axis=1)
            y_true = self.y_validation_true.argmax(axis=1)
        
        plt.style.use('seaborn')
        plt.figure(figsize = (20,6), dpi=dpi)
        plt.grid(True)
        plt.plot(y_pred[:num], color = 'lightcoral')
        plt.plot(y_true[:num], color = 'cornflowerblue', linewidth = 1)
        plt.title('{}_{}'.format(ticker, plot_name))
        plt.legend(['predict', 'true'])
#         if plot_name == 'Test':
#             plt.savefig('{}_{}_{}_{}.png'.format(ticker, plot_name, str(time_steps), str(batch_size)))
#         else:
#             plt.savefig('{}_{}_{}_{}.png'.format(ticker, plot_name, str(time_steps), str(batch_size)))


In [168]:
lstm_model = LSTM_Model()
lstm_model.build(time_steps = time_steps, data_dim = pipeline.x.shape[-1], output_dim = pipeline.y.shape[-1])
pipeline.model_training_testing(ticker, lstm_model)

train (13593, 50, 4) (13593, 3)
validation (1511, 50, 4) (1511, 3)
Train on 13593 samples, validate on 1511 samples
Epoch 1/100
13593/13593 [==============================] - 12s 864us/step - loss: 0.8630 - acc: 0.4804 - val_loss: 0.8244 - val_acc: 0.4950
Epoch 2/100
13593/13593 [==============================] - 10s 736us/step - loss: 0.8142 - acc: 0.4987 - val_loss: 0.7968 - val_acc: 0.5202
Epoch 3/100
13593/13593 [==============================] - 10s 746us/step - loss: 0.7911 - acc: 0.5312 - val_loss: 0.7834 - val_acc: 0.5341
Epoch 4/100
13593/13593 [==============================] - 10s 751us/step - loss: 0.7678 - acc: 0.5713 - val_loss: 0.7626 - val_acc: 0.5625
Epoch 5/100
13593/13593 [==============================] - 11s 806us/step - loss: 0.7525 - acc: 0.5878 - val_loss: 0.7540 - val_acc: 0.5639
Epoch 6/100
13593/13593 [==============================] - 11s 806us/step - loss: 0.7477 - acc: 0.5866 - val_loss: 0.7500 - val_acc: 0.5685
Epoch 7/100
13593/13593 [===================

/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


808


/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


20190117 accuracy:  0.5866336633663366
5342


...

/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


20190129 accuracy:  0.5851063829787234
2164


/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


20190130 accuracy:  0.583641404805915
2679
20190131 accuracy:  0.6129152668906308


/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [169]:
print(pipeline.x.shape, pipeline.y.shape)

(15154, 50, 4) (15154, 3)


# CNN

In [219]:
from keras.layers import Dense, Conv2D, Flatten, Dropout
from keras import regularizers

In [230]:
hidden_dim = 30
n_epochs = 100
batch_size = 128
activation = "tanh"
loss = 'categorical_crossentropy'
stop_patience=20

dpi=200


class CNN_Model():
    def __init__(self):
        self.model = Sequential()
        self.model_name = "CNN"
        return 
    
    def build(self, time_steps = time_steps, data_dim = 1, output_dim = 3):
        #add model layers
        self.model.add(Conv2D(28, kernel_size=2, activation='relu', input_shape=(time_steps, data_dim, 1)))
        self.model.add(Dropout(0.3))
        self.model.add(Conv2D(28, kernel_size=2, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
        self.model.add(Dropout(0.3))
        self.model.add(Flatten())
        self.model.add(Dense(output_dim, activation='softmax'))
        
        opt=keras.optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        self.model.compile(loss = loss, optimizer=opt, metrics=['accuracy']) 
        return self.model
    
    def fit(self, x_train, y_train, batch_size, epochs, validation_data, callbacks):
        return self.model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = validation_data, callbacks = callbacks)
    
    def predict(self, x):
        return self.model.predict(x)
    
    def reshape_dataset(self, x, y):
        if x is not None:
            if len(x.shape) == 3:
                x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
        if len(y.shape) == 1:
            y = y.reshape(y.shape[0],1)
        return x, y 


class Model_Functionalities():
    def __init__(self, model):
        self.model = model
    def train_test(self, x, y, plot = False):
        
        size = len(x)
        if size!=len(y):
            return None
        x = x[: batch_size * (size // batch_size)]
        y = y[: batch_size * (size // batch_size)]
        
        x, y = self.model.reshape_dataset(x, y)

        x_train, x_validation, y_train, y_validation= train_test_split(x, y, test_size = 0.1, shuffle = True)
        print('train', x_train.shape, y_train.shape)
        print('validation', x_validation.shape, y_validation.shape)
        
        
        if self.model.model_name == "LSTM" or  self.model.model_name == "CNN":
            early_stopping = EarlyStopping(monitor='val_loss', patience=stop_patience, mode="min", verbose=2, restore_best_weights=True)
            history = self.model.fit(x_train, y_train, batch_size = batch_size, epochs = n_epochs,
                                     validation_data=(x_validation, y_validation),callbacks=[early_stopping])
        else:
            self.model.fit(x_train, y_train)
        
        self.y_pred = self.model.predict(x_validation)
        self.y_validation_true = y_validation
        
        if plot == True:
            if y.shape[1] != 1:
                self.train_plot = self.view_accuracy(self.predict(x_train).argmax(axis=1), y_train.argmax(axis=1), 'Train')
                self.validation_plot = self.view_accuracy(self.predict(x_validation).argmax(axis=1), y_validation.argmax(axis=1), 'Validation')
            else:
                self.train_plot = self.view_accuracy(self.predict(x_train), y_train, 'Train')
                self.validation_plot = self.view_accuracy(self.predict(x_validation), y_validation, 'Validation')
        if self.model.model_name == "LSTM":
            return history

    def predict(self, x_validation):
        pred = self.model.predict(x_validation)
        return pred
    
    def view_accuracy(self, y_pred = None, y_true = None, plot_name = 'Test', num=100):
        if y_pred is None:
            y_pred = self.y_pred.argmax(axis=1)
            y_true = self.y_validation_true.argmax(axis=1)
        
        plt.style.use('seaborn')
        plt.figure(figsize = (20,6), dpi=dpi)
        plt.grid(True)
        plt.plot(y_pred[:num], color = 'lightcoral')
        plt.plot(y_true[:num], color = 'cornflowerblue', linewidth = 1)
        plt.title('{}_{}'.format(ticker, plot_name))
        plt.legend(['predict', 'true'])
#         if plot_name == 'Test':
#             plt.savefig('{}_{}_{}_{}.png'.format(ticker, plot_name, str(time_steps), str(batch_size)))
#         else:
#             plt.savefig('{}_{}_{}_{}.png'.format(ticker, plot_name, str(time_steps), str(batch_size)))


In [231]:
cnn_model = CNN_Model()
cnn_model.build(time_steps = time_steps, data_dim = pipeline.x.shape[-1], output_dim = pipeline.y.shape[-1])
pipeline.model_training_testing(ticker, cnn_model)

train (13593, 50, 4, 1) (13593, 3)
validation (1511, 50, 4, 1) (1511, 3)
Train on 13593 samples, validate on 1511 samples
Epoch 1/100
13593/13593 [==============================] - 4s 274us/step - loss: 0.8441 - acc: 0.5083 - val_loss: 0.8301 - val_acc: 0.5619
Epoch 2/100
13593/13593 [==============================] - 2s 119us/step - loss: 0.7914 - acc: 0.5605 - val_loss: 0.8095 - val_acc: 0.5659
Epoch 3/100
13593/13593 [==============================] - 2s 120us/step - loss: 0.7754 - acc: 0.5719 - val_loss: 0.7990 - val_acc: 0.5804
Epoch 4/100
13593/13593 [==============================] - 2s 120us/step - loss: 0.7651 - acc: 0.5779 - val_loss: 0.7846 - val_acc: 0.5890
Epoch 5/100
13593/13593 [==============================] - 2s 119us/step - loss: 0.7597 - acc: 0.5808 - val_loss: 0.7896 - val_acc: 0.5817
Epoch 6/100
13593/13593 [==============================] - 2s 129us/step - loss: 0.7580 - acc: 0.5767 - val_loss: 0.7849 - val_acc: 0.5745
Epoch 7/100
13593/13593 [===================

/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


808
20190117 accuracy:  0.5742574257425742
5342
20190118 accuracy:  0.6149382253837514
2570
20190122 accuracy:  0.5988326848249027
2944
20190123 accuracy:  0.6019021739130435
1759
20190124 accuracy:  0.572484366117112
1610
20190125 accuracy:  0.5832298136645963
1428
20190128 accuracy:  0.5777310924369747
1034
20190129 accuracy:  0.5831721470019342
2164
20190130 accuracy:  0.5637707948243993
2679
20190131 accuracy:  0.5834266517357223


# Random Forest

In [232]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, ClassifierMixin

In [233]:
class RandomForest():
    def __init__(self,max_depth=50, n_estimators=100, max_features=0.36, criterion = 'gini'):
        self.model = RandomForestClassifier(n_estimators=n_estimators, criterion = criterion, max_depth=max_depth,max_features=max_features)
        self.model_name = "Random_Forest"
    def fit(self,X,Y):
        return self.model.fit(X,Y)
    def predict(self,X):
        return self.model.predict(X)
    def reshape_dataset(self, x, y):
        if x is not None:
            if len(x.shape) == 3:
                x = x.reshape(x.shape[0], x.shape[1]*x.shape[2])
        if len(y.shape) == 1:
            y = y.reshape(y.shape[0],1)
        return x, y 

In [234]:
random_forest_model = RandomForest()
pipeline.model_training_testing(ticker, random_forest_model)

train (13593, 200) (13593, 3)
validation (1511, 200) (1511, 3)


/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


808
20190117 accuracy:  0.5618811881188119
5342
20190118 accuracy:  0.6042680643953575
2570
20190122 accuracy:  0.5622568093385214
2944
20190123 accuracy:  0.553328804347826
1759
20190124 accuracy:  0.533826037521319
1610
20190125 accuracy:  0.5391304347826087
1428
20190128 accuracy:  0.5301120448179272
1034
20190129 accuracy:  0.5435203094777563
2164
20190130 accuracy:  0.5619223659889094
2679
20190131 accuracy:  0.5718551698394924


# GradientBoost

In [173]:
from sklearn.ensemble import GradientBoostingClassifier

In [174]:
class GradientBoost():
    def __init__(self):
        self.model = GradientBoostingClassifier(n_estimators=100,max_depth=50,learning_rate=0.05)
        self.model_name = "GradientBoost"
    def fit(self,X,Y):
        return self.model.fit(X,Y)
    def predict(self,X):
        return self.model.predict(X)
    def reshape_dataset(self, x, y):
        if x is not None:
            if len(x.shape) == 3:
                x = x.reshape(x.shape[0], x.shape[1]*x.shape[2])
        if len(y.shape) != 1:
            y = np.where(y==1)[1]
            y = y.reshape(y.shape[0],1)
        return x, y 

In [175]:
gradient_boost_model=GradientBoost()
pipeline.model_training_testing(ticker, gradient_boost_model)

train (13593, 200) (13593, 1)
validation (1511, 200) (1511, 1)


/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, s

808
20190117 accuracy:  0.465444564258406
5342
20190118 accuracy:  0.43963022835926063
2570
20190122 accuracy:  0.4527048100652546
2944
20190123 accuracy:  0.45738938445179583
1759
20190124 accuracy:  0.4616870081940324
1610
20190125 accuracy:  0.46697966899425175
1428
20190128 accuracy:  0.4662188404773674
1034
20190129 accuracy:  0.4693851972958109
2164
20190130 accuracy:  0.46788354898336415
2679
20190131 accuracy:  0.46980378125191147


# XGBOOST

In [176]:
from xgboost import XGBClassifier

In [177]:
class XGBoost():
    def __init__(self):
        self.model =XGBClassifier(n_estimators=100, max_depth=50,learning_rate=0.1,reg_lambda=0.1, verbose=True)
        self.model_name = "XGBoost"
    def fit(self,X,Y):
        return self.model.fit(X,Y)
    def predict(self,X):
        return self.model.predict(X)
    def reshape_dataset(self, x, y):
        if x is not None:
            if len(x.shape) == 3:
                x = x.reshape(x.shape[0], x.shape[1]*x.shape[2])
        if len(y.shape) != 1:
            y = np.where(y==1)[1]
            y = y.reshape(y.shape[0],1)
        return x, y 

In [178]:
xgboost_model=XGBoost()
pipeline.model_training_testing(ticker, xgboost_model)

train (13593, 200) (13593, 1)
validation (1511, 200) (1511, 1)


/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/xunyingluo/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


808
20190117 accuracy:  0.47681140329379473
5342
20190118 accuracy:  0.4400565175748899
2570
20190122 accuracy:  0.45910309013005496
2944
20190123 accuracy:  0.45345763583264415
1759
20190124 accuracy:  0.46595677359448573
1610
20190125 accuracy:  0.48326029088383937
1428
20190128 accuracy:  0.4760521855801144
1034
20190129 accuracy:  0.48628450852822225
2164
20190130 accuracy:  0.47385293203180257
2679
20190131 accuracy:  0.473853361016051


#  Tick Factor

In [ ]:
# TICK FACTOR
# only update if it's a trade
# if message_type == 't':
#     # calc the tick
#     this_tick = np.sign(last_price - prev_price)
#     if this_tick == 0:
#         this_tick = prev_tick

#     # now calc the tick
#     if tick_factor == 0:
#         tick_factor = this_tick
#     else:
#         tick_factor = (tick_ema_alpha * this_tick) + (1 - tick_ema_alpha) * tick_factor

#         # store the last tick
#     prev_tick = this_tick

for test_date in test_date_list:
    df = pd.read_csv(data_dir+'trades_{}_{}.csv'.format(test_date, ticker),index_col=[0],parse_dates=[0])
    
    df["tick_test"]=(df["trade_px"]-df["trade_px"].shift(1)).apply(lambda x: 1 if x>0. else (-1. if x<0 else np.nan))
    df.fillna(method="ffill",inplace=True)
    
    df["tick_factor"]=df["tick_test"].ewm(span=20).mean()
    df.dropna(axis=0,inplace=True)
    
    mysign = lambda x: 0 if abs(x)<1e-5 else (1 if x>0 else -1)
    df["predict"]=df["tick_factor"].apply(mysign)
    df["real_movement"]=(df["trade_px"].rolling(nforward).mean().shift(-nforward)-df["trade_px"]).apply(mysign)
    
    df.dropna(axis=0,inplace=True)
    acc=np.mean(df["predict"]==df["real_movement"])
    print("Accuracy of {}".format(test_date),acc)


In [ ]:
sum=pd.Timedelta(0)
count=0
for test_date in test_date_list:
    df = pd.read_csv(data_dir+'trades_{}_{}.csv'.format(test_date, ticker),index_col=[0],parse_dates=[0])
    timestamp=pd.DataFrame({"trade_time":df.index})
    dt=timestamp["trade_time"].shift(-nforward)-timestamp["trade_time"]
    dt.dropna(axis=0,inplace=True)
    dt.apply(lambda x:x.seconds).hist()
    sum+=dt.sum()
    count+=dt.shape[0]
print("Average time interval between {} trades is".format(nforward),sum/count)